In [1]:
import numpy as np
import tifffile as tiff
import pandas as pd
import create_mask_for_region
import os

In [2]:
# takes an inputpath as a location of the files
# uses fnames as ways to identify files
atlas_imgpath   = r"\\10.159.50.7\Analysis2\Ken\ClearMap\clearmap_ressources_mouse_brain\ClearMap_ressources\Regions_annotations\Kim_ref_adult_FP-label_v4.0.tif"
atlas_img       = tiff.imread(atlas_imgpath)


atlas_dfpath    = r"\\10.159.50.7\Analysis2\Ken\ClearMap\clearmap_ressources_mouse_brain\ClearMap_ressources\Regions_annotations\atlas_info_KimRef_FPbasedLabel_v4.0.csv"
atlas_df        = pd.read_csv(atlas_dfpath)


In [3]:
# calculate size


In [4]:
ids,counts = np.unique(atlas_img,return_counts = True)

In [5]:
atlas_df = atlas_df.merge(pd.DataFrame({'id':ids,'size_by_voxel':counts}),left_on = 'id',right_on = 'id',how = 'outer')

In [6]:
atlas_df['newsize_by_voxel'] = 0
for iid in atlas_df.id:
    tdf = create_mask_for_region.get_subregions(atlas_df, iid, return_original = True)
    atlas_df.loc[atlas_df.id == iid,'newsize_by_voxel'] = np.sum(tdf.size_by_voxel)

KeyboardInterrupt: 

In [ ]:
atlas_df['newsize'] = atlas_df['newsize_by_voxel'] * (atlas_res[0] * atlas_res[1] * atlas_res[2])*((10**-3)**3) # one voxel volume

In [178]:
atlas_df = atlas_df.sort_values(by='structure_order')

In [ ]:
atlas_df.to_csv(r"\\10.159.50.7\Analysis2\Ken\ClearMap\clearmap_ressources_mouse_brain\ClearMap_ressources\Regions_annotations\atlas_info_KimRef_FPbasedLabel_v4.0_with_size.csv",index = False)

# transfer useful columns from previous mouse brain atlas dataframes

In [7]:
'''atlas_df = pd.read_csv(r"\\10.159.50.7\Analysis2\Ken\ClearMap\clearmap_ressources_mouse_brain\ClearMap_ressources\Regions_annotations\atlas_info_KimRef_FPbasedLabel_v4.0_brain_with_size.csv",index_col = False)
old_atlas_df = pd.read_csv(r"\\10.159.50.7\Analysis2\Ken\ClearMap\clearmap_ressources_mouse_brain\ClearMap_ressources\Regions_annotations\atlas_info_KimRef_FPbasedLabel_v2.9_brain_with_size_with_curated_with_cleaned_acronyms.csv",index_col = False)
atlas_df = atlas_df.merge(old_atlas_df[['id','ancestor_default_acronym','cleaned_acronym']],left_on = 'id',right_on = 'id',how = 'outer').sort_values(by = 'structure_order')
atlas_df.to_csv(r"\\10.159.50.7\Analysis2\Ken\ClearMap\clearmap_ressources_mouse_brain\ClearMap_ressources\Regions_annotations\atlas_info_KimRef_FPbasedLabel_v4.0_brain_with_size_with_curated_with_cleaned_acronyms.csv",index = False)'''

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\stuberadmin\AppData\Local\Temp;\ipykernel_53468\4268928490.py:1: SyntaxWarning: invalid escape sequence '\A'
  '''atlas_df = pd.read_csv(r"\\10.159.50.7\Analysis2\Ken\ClearMap\clearmap_ressources_mouse_brain\ClearMap_ressources\Regions_annotations\atlas_info_KimRef_FPbasedLabel_v4.0_brain_with_size.csv",index_col = False)


'atlas_df = pd.read_csv(r"\\10.159.50.7\\Analysis2\\Ken\\ClearMap\\clearmap_ressources_mouse_brain\\ClearMap_ressources\\Regions_annotations\x07tlas_info_KimRef_FPbasedLabel_v4.0_brain_with_size.csv",index_col = False)\nold_atlas_df = pd.read_csv(r"\\10.159.50.7\\Analysis2\\Ken\\ClearMap\\clearmap_ressources_mouse_brain\\ClearMap_ressources\\Regions_annotations\x07tlas_info_KimRef_FPbasedLabel_v2.9_brain_with_size_with_curated_with_cleaned_acronyms.csv",index_col = False)\natlas_df = atlas_df.merge(old_atlas_df[[\'id\',\'ancestor_default_acronym\',\'cleaned_acronym\']],left_on = \'id\',right_on = \'id\',how = \'outer\').sort_values(by = \'structure_order\')\natlas_df.to_csv(r"\\10.159.50.7\\Analysis2\\Ken\\ClearMap\\clearmap_ressources_mouse_brain\\ClearMap_ressources\\Regions_annotations\x07tlas_info_KimRef_FPbasedLabel_v4.0_brain_with_size_with_curated_with_cleaned_acronyms.csv",index = False)'

In [8]:
atlas_df = pd.read_csv(r"\\10.159.50.7\Analysis2\Ken\ClearMap\clearmap_ressources_mouse_brain\ClearMap_ressources\Regions_annotations\atlas_info_KimRef_FPbasedLabel_v4.0_brain_with_size_with_curated_with_cleaned_acronyms.csv",index_col = False)

In [9]:

def count_labels(inputpath,channel_key,atlas_img,atlas_df):
    # reads numpy files that were exported from ClearMap
    cells = np.load(os.path.join(inputpath,f"{channel_key}_transformed_to_Atlas.npy"))
    if not 'stitched' not in channel_key:
        weights = np.load(os.path.join(inputpath,f"{channel_key}_intensities.npy"))

    # conver to integers
    tcells = cells.astype('int')

    # filter cells out side the range
    tcells = tcells[(tcells[:,0] < atlas_img.shape[2]) & \
    (tcells[:,1] < atlas_img.shape[1]) & \
    (tcells[:,2] < atlas_img.shape[0])]

    cell_labels = atlas_img[tcells[:,2],tcells[:,1],tcells[:,0]]

    # count labels and get counts and ids in the same order as labels in atlas_img  (we assume atlas_img labels are in ascending order)  (this is a bit tricky because labels can be repeated in atlas_img)  but we can assume that the order is correct because the atlas_df is sorted by id.
    ids,counts = np.unique(cell_labels,return_counts=  True)
    # create a table with counts and ids  (this assumes that the order of ids in atlas_df is the same as in atlas_img)
    table = np.zeros(ids.shape, dtype=[('id','int64'),('counts','f8')])
    table["id"] = ids;
    table["counts"] = counts;
        
    counts_df = atlas_df.set_index('id').join(pd.DataFrame(table).set_index('id'))
    counts_df['raw_cell_intensity'] = 0.0
    counts_df['raw_cell_size'] = 0.0
    
    if not 'stitched' not in channel_key:
        # add raw_cell_intensity and size info
        for i in ids:
            raw_cell_intensity,raw_cell_size = weights[np.where(cell_labels == i)].sum(axis = 0)[[0,-1]]
            counts_df.loc[i,'raw_cell_intensity'] = raw_cell_intensity
            counts_df.loc[i,'raw_cell_size'] = raw_cell_size

    # update the counts    
    counts_df = update_counts(counts_df.reset_index())

    return counts_df

In [10]:
def update_counts(df):
    df['newcounts'] = 0.0
    df['newcellintensity'] = 0.0
    df['newcellsize'] = 0.0
    for iid in df.id:
        #print(iid)
        tdf = create_mask_for_region.get_subregions(df, iid, return_original = True)
        df.loc[df.id == iid,'newcounts'] = np.sum(tdf.counts)
        df.loc[df.id == iid,'newcellintensity'] = np.sum(tdf['raw_cell_intensity'].values)
        df.loc[df.id == iid,'newcellsize'] = np.sum(tdf['raw_cell_size'].values)
    # normalize counts and cell intensity
    df['normalized_cellintensity'] = df['newcellintensity'] / df['newcounts']
    df['normalized_cellsize'] = df['newcellsize'] / df['newcounts']

    return df

In [9]:
rootpath = r"Y:\SmartSPIM2\Ken\Fentanyl"
overwriteflag = False
channel_key = 'Ex_639_Ch2_stitched'
#channels = ['overlap_strict',]
channels = ['Ex_639_Ch2_stitched',]
fnames = [f for f in os.listdir(rootpath) if 'Destripe_DONE' in f]
for channel_key in channels:
    print(f'Processing {channel_key}')
    for fname in fnames:
        if 'BAD' in fname:
            continue
        print(f'Processing {fname}')
        inputpath = os.path.join(rootpath,fname)
        outputpath = os.path.join(inputpath,f'{channel_key}_Annotated_counts_with_leaf.csv')
        # skip if outputpath already exists and the overwrite flag is false
        if os.path.exists(outputpath) and not overwriteflag:
            print("file already exists")
            #continue
        counts_df = count_labels(inputpath,channel_key,atlas_img,atlas_df)
        counts_df.to_csv(outputpath,index = False)

Processing Ex_639_Ch2_stitched
Processing 20250218_13_01_01_lynn_Fentanyl_F02_Destripe_DONE
file already exists
Processing 20250219_14_06_09_lynn_Fentanyl_F08_Destripe_DONE
file already exists
Processing 20250218_12_37_35_lynn_Fentanyl_S02_Destripe_DONE
file already exists
Processing 20250203_16_45_05_lynn_Fentanyl_S01_Destripe_DONE
file already exists
Processing 20250218_14_10_11_lynn_Fentanyl_F03_Destripe_DONE
file already exists
Processing 20250219_12_09_03_lynn_Fentanyl_F05_Destripe_DONE
file already exists
Processing 20250218_16_04_27_lynn_Fentanyl_S04_Destripe_DONE
file already exists
Processing 20250218_14_33_39_lynn_Fentanyl_S03_Destripe_DONE
file already exists
Processing 20250218_16_27_02_lynn_Fentanyl_F04_Destripe_DONE
file already exists
Processing 20250203_16_23_47_lynn_Fentanyl_F01_Destripe_DONE
file already exists
Processing 20250219_11_46_27_lynn_Fentanyl_F06_Destripe_DONE
file already exists
Processing 20250219_13_42_16_lynn_Fentanyl_F07_Destripe_DONE
file already exis

# merge the files into one master file

In [11]:
from datetime import datetime

In [12]:
# set paths
result_paths = r"\\10.159.50.7\analysis2\Ken\LSMS\Opioid"
result_paths2 = r"\\10.158.246.229\DataCommon\SmartSPIM2\Ken\Fentanyl"
rootpath = r"\\10.159.50.7\LabCommon\Ken\data\Fentanyl"
metapath = os.path.join(rootpath,"meta")
analysis_resultpath = os.path.join(rootpath,"result")
analysis_figurepath = os.path.join(rootpath,"figure")
for path in [analysis_resultpath,analysis_figurepath]:
    if not os.path.exists(path):
        os.mkdir(path)

# load meta info of the files
metadf = pd.read_csv(os.path.join(metapath,"Fentanyl_meta.csv"),index_col= False)
# temporary drop of A7 due to missing data
metadf = metadf[metadf.Usable].reset_index(drop = True)
#metadf = metadf[metadf.Usable].reset_index(drop = True)
#metadf = metadf[metadf.Condition.str.contains('Fentanyl')].reset_index(drop = True)
# load brain atlas to register
#atlas_df = pd.read_csv(r"\\10.159.50.7\Analysis2\Ken\ClearMap\clearmap_ressources_mouse_brain\ClearMap_ressources\Regions_annotations\atlas_info_KimRef_FPbasedLabel_v2.9_brain_with_size_with_curated.csv",index_col = False)
metacolumns = ['id','acronym','parent_acronym','parent_id','structure_order']
channels = ['Ex_639_Ch2_stitched']
#channels = ['overlap_strict','Ex_561_Ch1_stitched','Ex_639_Ch2_stitched']
#channels = ['Ex_561_Ch1_stitched',]
# retrieve list of files
fnames =  [f for f in metadf.fname.values if 'DONE' in f and 'BAD' not in f]
print(len(fnames))

19


In [13]:
metadf

,ID,fname,DOB,Cage,Condition,Sex,BW,Date_Injection_1,Date_Perfusion,Notes,Staining_Batch,Drug,Chronic,Acute,Usable
0,S1,20240227_23_20_32_OP_S1I_Destripe_DONE,11/9/2023,287527.0,Saline,F,21.6,1/30/2024,2/6/2024,NaN,1,Saline,False,False,True
1,S2,20240227_23_00_56_OP_S2J_Destripe_DONE,11/9/2023,287528.0,Saline,M,27.0,1/30/2024,2/6/2024,bad perfusion,1,Saline,False,False,True
2,S3,20240228_10_52_10_OP_S3L_Destripe_DONE,11/9/2023,287528.0,Saline,M,27.3,1/30/2024,2/6/2024,NaN,1,Saline,False,False,True
3,S4,20240228_11_13_31_OP_S4K_Destripe_DONE,8/14/2023,267319.0,Saline,M,37.0,1/30/2024,2/6/2024,NaN,1,Saline,False,False,True
4,S5,20240403_10_59_33_OP_S5M_Destripe_DONE,9/1/2023,277640.0,Saline,M,31.0,1/30/2024,2/6/2024,NaN,2,Saline,False,False,True
5,S6,20240403_11_19_31_OP_S6N_Destripe_DONE,9/1/2023,277640.0,Saline,M,28.0,1/30/2024,2/6/2024,NaN,2,Saline,False,False,True
6,S7,20240403_12_02_34_OP_S7O_Destripe_DONE,11/9/2023,287526.0,Saline,M,26.5,1/30/2024,2/6/2024,NaN,2,Saline,False,False,True
7,S8,20240403_12_21_49_OP_S8P_Destripe_DONE,11/9/2023,287526.0,Saline,M,26.0,1/30/2024,2/6/2024,NaN,2,Saline,False,False,True
8,mF08,20250219_14_06_09_lynn_Fentanyl_F08_Destripe_DONE,9/14/2024,NaN,Withdrawal_Fentanyl,M,27.0,11/1/2024,11/27/2024,NaN,5,Fentanyl,True,False,True
9,mF07,20250219_13_42_16_lynn_Fentanyl_F07_Destripe_DONE,9/14/2024,NaN,Withdrawal_Fentanyl,F,15.8,11/1/2024,11/27/2024,NaN,5,Fentanyl,True,False,True


In [14]:
# read the dataframe with size
whole_atlas_dfpath = r"\\10.159.50.7\Analysis2\Ken\ClearMap\clearmap_ressources_mouse_brain\ClearMap_ressources\Regions_annotations\atlas_info_KimRef_FPbasedLabel_v4.0_brain_with_size_with_curated_with_cleaned_acronyms.csv"

whole_atlas_df = pd.read_csv(whole_atlas_dfpath)

In [17]:
for channel_key in channels:
    for idx,fname in enumerate(fnames):
        #if fname in np.unique(long_merge_df.fname):
        #    continue

        #dfpath = [os.path.join(result_paths, fname,f) for f in os.listdir(os.path.join(rootpath, fname)) if '_Annotated_counts_with_leaf.csv' in f][0]
        # load metainfo from each files
        ID,fname,DOB,Condition,Sex,Staining_Batch,BW,Date_Perfusion = \
            metadf.loc[metadf.fname == fname,['ID','fname','DOB','Condition','Sex','Staining_Batch','BW','Date_Perfusion']].values[0]
        print(f"Processing {ID}")
        try:
            dfpath = os.path.join(result_paths2,fname,f'{channel_key}_Annotated_counts_with_leaf.csv')
            df = pd.read_csv(dfpath,index_col= False)
        except:
            dfpath = os.path.join(result_paths,fname,f'{channel_key}_Annotated_counts_with_leaf.csv')
            df = pd.read_csv(dfpath,index_col= False)
        # load the cleaned result
        
        df['newcounts'] = 0
        for iid in df.id:
            tdf = create_mask_for_region.get_subregions(df, iid, return_original = True)
            df.loc[df.id == iid,'newcounts'] = np.sum(tdf.counts)
            
        # Subset the dataframe to only contain brains
        df = whole_atlas_df.merge(df[['acronym','counts','newcounts']],left_on = ['acronym'],right_on = ['acronym'],how = 'inner')

        # add meta data to the dataframe
        df['ID'] = ID
        df['fname'] = fname
        df['Condition'] = Condition
        #df['Genotype'] = Genotype
        df['Sex'] = Sex
        df['BW'] = BW
        df['Age'] = (datetime.strptime( Date_Perfusion, '%m/%d/%Y') - datetime.strptime( DOB, '%m/%d/%Y')).days
        df['density'] = df['newcounts']/df['newsize']
        df['density'] =  df['density'].fillna(0)
        df['Staining_Batch'] = Staining_Batch

        #simple_df = df[['id','parent_id','name','acronym','parent_acronym','density']]
        #simple_df = simple_df.rename(columns = {'density':Condition + '_' + ID})

        

        # Save the updated dataframe
        #df.to_csv(os.path.join(result_paths,fname,'Annotated_counts_clean_with_density.csv'))
        # concatenate the dataframe to one
        if idx == 0:
            long_merge_df = df
        else:
            long_merge_df = pd.concat([long_merge_df,df],axis = 0).reset_index(drop = True)
    long_merge_df.to_csv(os.path.join(analysis_resultpath,f'{channel_key}_long_merge_Annotated_counts_with_leaf_with_density.csv'),index = False)

Processing S1
Processing S2
Processing S3
Processing S4
Processing S5
Processing S6
Processing S7
Processing S8
Processing mF08
Processing mF07
Processing mF05
Processing mF06
Processing mS04
Processing mS03
Processing mF03
Processing mF02
Processing mS02
Processing mS01
Processing mF01


In [57]:
DOB

nan

In [52]:
for channel_key in channels:
    for idx,fname in enumerate(fnames):
        #if fname in np.unique(long_merge_df.fname):
        #    continue
        dfpath = os.path.join(result_paths, fname,f'{channel_key}_Annotated_counts_with_leaf.csv')
        #dfpath = [os.path.join(result_paths, fname,f) for f in os.listdir(os.path.join(rootpath, fname)) if '_Annotated_counts_with_leaf.csv' in f][0]
        # load metainfo from each files
        ID,fname,DOB,Condition,Sex,Staining_Batch,BW,Date_Perfusion = metadf.loc[metadf.fname == fname,['ID','fname','DOB','Condition','Sex','Staining_Batch','BW','Date_Perfusion']].values[0]
        print(f"Processing {ID}")
        # load the cleaned result
        df = pd.read_csv(dfpath,index_col= False)
        df['newcounts'] = 0
        for iid in df.id:
            tdf = create_mask_for_region.get_subregions(df, iid, return_original = True)
            df.loc[df.id == iid,'newcounts'] = np.sum(tdf.counts)
            
        # Subset the dataframe to only contain brains
        df = whole_atlas_df.merge(df[['acronym','counts','newcounts']],left_on = ['acronym'],right_on = ['acronym'],how = 'inner')

        # add meta data to the dataframe
        df['ID'] = ID
        df['fname'] = fname
        df['Condition'] = Condition
        df['Sex'] = Sex
        df['BW'] = BW
        df['Age'] = (datetime.strptime( Date_Perfusion, '%m/%d/%Y') - datetime.strptime( DOB, '%m/%d/%Y')).days
        df['density'] = df['newcounts']/df['newsize']
        df['density'] =  df['density'].fillna(0)
        df['Staining_Batch'] = Staining_Batch

        #simple_df = df[['id','parent_id','name','acronym','parent_acronym','density']]
        #simple_df = simple_df.rename(columns = {'density':Condition + '_' + ID})

        

        # Save the updated dataframe
        #df.to_csv(os.path.join(result_paths,fname,'Annotated_counts_clean_with_density.csv'))
        # concatenate the dataframe to one
        if idx == 0:
            long_merge_df = df
        else:
            long_merge_df = pd.concat([long_merge_df,df],axis = 0).reset_index(drop = True)
    long_merge_df.to_csv(os.path.join(analysis_resultpath,f'{channel_key}_long_merge_Annotated_counts_with_leaf_with_density.csv'),index = False)

Processing SM1
Processing SM2
Processing SM3
Processing MM1
Processing MM2
Processing MM3
Processing CM1
Processing CM2
Processing CM3
Processing MM4
Processing MM5
Processing MM6
Processing SM4
Processing SM5
Processing CM4
Processing CM5
Processing MW1
Processing MW2
Processing MW3
Processing MW4
Processing MW5
Processing MW6
Processing AC1
Processing AC4
Processing AC3
Processing AC6
Processing AC8
Processing AC7
Processing AC9
Processing SM1
Processing SM2
Processing SM3
Processing MM1
Processing MM2
Processing MM3
Processing CM1
Processing CM2
Processing CM3
Processing MM4
Processing MM5
Processing MM6
Processing SM4
Processing SM5
Processing CM4
Processing CM5
Processing MW1
Processing MW2
Processing MW3
Processing MW4
Processing MW5
Processing MW6
Processing AC1
Processing AC4
Processing AC3
Processing AC6
Processing AC8
Processing AC7
Processing AC9


In [42]:
long_merge_df

,acronym,id,name,red,green,blue,structure_order,parent_id,parent_acronym,collapse,...,counts,newcounts,ID,fname,Condition,Sex,BW,Age,density,Staining_Batch
0,CH,567,Cerebrum,176,240,255,200,8,grey,NaN,...,NaN,2398,SM1,20240407_18_09_25_OPTRAP_SM1_Destripe_DONE,Sal-AcM,F,27,46,9.007539,1
1,CTX,688,Cerebral cortex,176,255,184,300,567,CH,NaN,...,NaN,2036,SM1,20240407_18_09_25_OPTRAP_SM1_Destripe_DONE,Sal-AcM,F,27,46,9.614320,1
2,CTXpl,695,Cortical plate,112,255,112,400,688,CTX,NaN,...,NaN,1762,SM1,20240407_18_09_25_OPTRAP_SM1_Destripe_DONE,Sal-AcM,F,27,46,8.709499,1
3,Isocortex,315,Isocortex,112,255,113,500,695,CTXpl,NaN,...,NaN,949,SM1,20240407_18_09_25_OPTRAP_SM1_Destripe_DONE,Sal-AcM,F,27,46,7.710603,1
4,FrA,184,Frontal association cortex,112,255,113,600,315,Isocortex,NaN,...,NaN,0,SM1,20240407_18_09_25_OPTRAP_SM1_Destripe_DONE,Sal-AcM,F,27,46,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40334,RMg,206,RaphÈ magnus nucleus,255,176,193,96900,379,MY-sat,NaN,...,NaN,2,AC9,20241028_12_38_47_OPTRAP_AC9_Destripe_DONE,AcM-ChM,F,17,65,6.792555,3
40335,RMg_leaf,1971,"RaphÈ magnus nucleus, leaf",255,176,193,96900,206,RMg,NaN,...,2.0,2,AC9,20241028_12_38_47_OPTRAP_AC9_Destripe_DONE,AcM-ChM,F,17,65,8.163265,3
40336,RIP,2212,RaphÈ interpositus nucleus,255,176,193,96910,206,RMg,NaN,...,NaN,0,AC9,20241028_12_38_47_OPTRAP_AC9_Destripe_DONE,AcM-ChM,F,17,65,0.000000,3
40337,RPa,230,RaphÈ pallidus nucleus,255,176,195,97000,379,MY-sat,NaN,...,NaN,0,AC9,20241028_12_38_47_OPTRAP_AC9_Destripe_DONE,AcM-ChM,F,17,65,0.000000,3


In [32]:
analysis_resultpath

'\\\\10.159.50.7\\LabCommon\\Ken\\data\\Fentanyl\\result'